In [1]:
pip install kagglehub


In [2]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shayanfazeli/heartbeat")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\shree\.cache\kagglehub\datasets\shayanfazeli\heartbeat\versions\1


In [4]:
train = pd.read_csv(os.path.join(path,'mitbih_train.csv'), header = None)
test = pd.read_csv(os.path.join(path,'mitbih_test.csv'), header = None)

train.head()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


CNN-1D

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


In [6]:
pip install tensorflow

In [7]:
X_train = train.iloc[:, :-1].values
y_train = train.iloc[:, -1].values

X_test = test.iloc[:, :-1].values
y_test = test.iloc[:, -1].values



Normalizing ECG Signals

In [8]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test =  scaler.transform(X_test)

Data Leakage if we do fit on X_test. X is used inside the model so only that is scaled not y

In [9]:
X_train.shape

(87554, 187)

In [10]:
y_train.shape

(87554,)

In [11]:
X_train = X_train[...,np.newaxis]
X_test = X_test[..., np.newaxis]


In [12]:
X_train.shape

(87554, 187, 1)

One Hot Encoding is done to maintain equality amog labels 0 to 4. 4 is not greater than 0

In [13]:

from tensorflow.keras.utils import to_categorical

num_classes = len(np.unique(y_train))

y_train= to_categorical(y_train, num_classes)
y_test= to_categorical(y_test, num_classes)

c:\Users\shree\anac\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [14]:
y_train.shape

(87554, 5)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D , MaxPooling1D , Flatten , Dense , Dropout



In [16]:
model = Sequential([
    Conv1D(32, kernel_size = 5, activation ="relu", 
           input_shape =(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),

    Conv1D(64, kernel_size =5 , activation ="relu"),
    MaxPooling1D(pool_size =2),

    Conv1D(128, kernel_size=3, activation ="relu"),
    Flatten(),

    Dense(128, activation = "relu"),
    Dropout(0.5),

    Dense(num_classes, activation ="softmax")       
           
           
    ])

c:\Users\shree\anac\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy",
    metrics=["accuracy"]
)

In [19]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size = 64,
    validation_split=0.2,
    verbose=1
)

Epoch 1/30
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 116s 78ms/step - accuracy: 0.9993 - loss: 0.0030 - val_accuracy: 0.1387 - val_loss: 112.3543
Epoch 2/30
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 76s 68ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.1387 - val_loss: 112.3543
Epoch 3/30
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 69s 62ms/step - accuracy: 1.0000 - loss: 1.1914e-11 - val_accuracy: 0.1387 - val_loss: 112.3550
Epoch 4/30
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 88s 80ms/step - accuracy: 1.0000 - loss: 1.7019e-12 - val_accuracy: 0.1387 - val_loss: 112.3550
Epoch 5/30
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 98s 39ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.1387 - val_loss: 112.3550
Epoch 6/30
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 39s 35ms/step - accuracy: 1.0000 - loss: 1.1914e-11 - val_accuracy: 0.1387 - val_loss: 112.3581
Epoch 7/30
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 60s 54ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.1387 - val_loss: 112.3581
Epoch 8/30
1095/1095 ━━━━━━━━━━━━━━━━

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)


The Kaggle ECG heartbeat dataset contains pre-segmented and normalized heartbeat signals derived from MIT-BIH. The CNN model was trained on beat-level ECG segments rather than raw continuous ECG recordings.